# Datawiser API — Quickstart

This notebook walks through each endpoint of the `datawiserai` Python client.

**Prerequisites:**
```bash
pip install 'datawiserai[pandas]'
```

Set your API key as an environment variable (`DATAWISER_API_KEY`) or paste it below.

In [1]:
import os
import datawiserai as dw

API_KEY = os.environ.get("DATAWISER_API_KEY", "your_api_key_here")
TICKER = "OLP"

client = dw.Client(api_key=API_KEY)
print(f"datawiserai v{dw.__version__}  —  connected")

datawiserai v0.1.0  —  connected


## Universe

Discover which tickers are available for a given endpoint.

In [ ]:
u = client.universe("free-float")
print(f"Available tickers: {u.tickers}")
print(f"'{TICKER}' in universe: {TICKER in u}")
u.to_dataframe()

## Free Float

Time-series of free-float factor, percentage, shares outstanding, and excluded shares.

In [ ]:
ff = client.free_float(TICKER)
print(f"ticker={ff.ticker}  security_id={ff.security_id}  events={len(ff)}")

latest = ff.latest()
print(f"Latest: {latest.as_of}  factor={latest.free_float_factor:.4f}  pct={latest.free_float_pct:.2f}%")

ff.to_dataframe()

## Shares Outstanding

Share-count observations sourced from 10-K, 10-Q, and other filings.

In [ ]:
so = client.shares_outstanding(TICKER)
print(f"ticker={so.ticker}  events={len(so)}")

latest_so = so.latest()
print(f"Latest: {latest_so.as_of}  type={latest_so.share_type}  shares={latest_so.shares:,.0f}")

so.to_dataframe()

## Reference Data

Identifiers, company information, and security metadata.

In [ ]:
ref = client.reference(TICKER)
print(f"ticker={ref.ticker}  company={ref.company_name}")
print(f"cik={ref.cik}  lei={ref.lei}  mic={ref.mic}  is_primary={ref.is_primary}")
print(f"\nCompany info: {ref.company_info}")
print(f"\nSecurity info: {ref.security_info}")

## Free Float Events — High-Level Summary

One row per event date with aggregate metrics.  
`is_rebal` is **True** when the event is a rebalance (e.g. new **DEF 14A** proxy).

In [ ]:
ffe = client.free_float_events(TICKER)
df_events = ffe.to_event_summary_dataframe()
df_events.head(10)

## Free Float Events — Flat Owner Summary

One row per owner per event date (top-level component fields only).

In [ ]:
df_owners = ffe.to_dataframe()
print(f"owner-rows={len(df_owners)}  distinct dates={len(ffe.dates())}")
df_owners[["as_of", "name", "shares", "delta_shares", "entity_type", "is_officer"]].head(10)

## Free Float Events — Detail Drill-Down

Full nested ownership structure: `ev.owner(owner_id)` returns an **OwnerDetail** with typed `components` (List[Component]), `restrictions` (List[Restriction]), `options` (List[Option]), and `event_details` (EventDetails or None).

In [ ]:
detail = client.free_float_events_detail(TICKER)
print(f"Event dates: {detail.dates[:5]}  ...\n")

ev = next(e for e in detail.events if e.is_rebal)
print(f"Event {ev.as_of} — {len(ev.owner_ids)} owners:")
for oid, name in list(ev.owner_names.items())[:5]:
    owner = ev.owner(oid)  # OwnerDetail: .components, .restrictions, .options, .event_details
    print(
        f"  {owner.name:35s}  shares={owner.shares:>12,.2f}  "
        f"components={len(owner.components)}  "
        f"restrictions={len(owner.restrictions)}  "
        f"options={len(owner.options)} "
        f"delta={ev.owner_delta(oid)}"
    )

## Extra: TSLA — Large Free-Float Move (2025-08-03)

Find events with large moves (e.g. |delta_fff_bps| > 10), then drill into a specific date and owner (e.g. restricted stock).

In [ ]:
df_tsla = client.free_float_events("TSLA").to_event_summary_dataframe()
large = df_tsla[df_tsla["delta_fff_bps"].abs() > 10]
print("Rows with |delta_fff_bps| > 10:")
display(df_tsla.loc[large.index[:5], ["as_of", "ff_factor", "delta_fff_bps", "delta_shares", "is_rebal"]])

detail_tsla = client.free_float_events_detail("TSLA")
ev_tsla = detail_tsla.by_date("2025-08-03")

print(f"\nDrill-down ev.delta for 2025-08-03: {ev_tsla.delta}")
owner_tsla = ev_tsla.owner("TSLATgVaxgiMn")
if owner_tsla.event_details:
    print(f"  owner.event_details.instrument_subtype: {owner_tsla.event_details.instrument_subtype}")
    print(f"  owner.event_details.notes: {(owner_tsla.event_details.notes or '')[:200]}...")

## Extra: TSLA — CEO Performance Award (2025-11-06)

Large negative move from CEO performance award (restricted stock) vesting.

In [ ]:
ev_tsla2 = detail_tsla.by_date("2025-11-06")

print(f"Event {ev_tsla2.as_of}: delta_ff_factor={ev_tsla2.delta_ff_factor:.6f}  delta_shares={ev_tsla2.delta_shares:,.0f}")
owner_tsla2 = ev_tsla2.owner("TSLATgVaxgiMn")

if owner_tsla2.event_details:
    print(f"  instrument_subtype: {owner_tsla2.event_details.instrument_subtype}")
    print(f"  notes (first 300 chars): {(owner_tsla2.event_details.notes or '')[:300]}...")

## Extra: AMZN — MacKenzie Scott (rebalance)

Private holder visible at proxy time; use a rebalance event and `owner_from_name` + `owner_delta`.

In [ ]:
detail_amzn = client.free_float_events_detail("AMZN")
ev_amzn = next((e for e in detail_amzn.events if e.is_rebal), None)

owner_ms = ev_amzn.owner_from_name("MacKenzie Scott")
oid_ms = owner_ms.owner_identity_id
print(f"{owner_ms.name:35s}  shares={owner_ms.shares:>12,.2f}  components={len(owner_ms.components)}  delta={ev_amzn.owner_delta(oid_ms)}")
